In [0]:
# Utility Functions
import json
import requests
import logging
from datetime import datetime, timedelta

def getWeatherData(latStr, lonStr, unitsStr, appIdStr, excludeStr):
    """
    Get weather data via the OpenWeatherMap API.
    
    Returns: Spark DataFrame
    
    """
    assert type(latStr) == str
    assert type(lonStr) == str
    assert type(unitsStr) == str
    assert type(appIdStr) == str
    assert type(excludeStr) == str
    url = "https://api.openweathermap.org/data/2.5/onecall?lat=" + latStr + "&lon=" + lonStr + "&units=" + unitsStr + "&exclude=" + excludeStr + "&appid=" + appIdStr
    
    myResponse = requests.get(url)
    
    if myResponse.ok:
        jDataRaw = json.loads(myResponse.content)
        jDataStr = json.dumps(jDataRaw)
        jDataList = []
        jDataList.append(jDataStr)
        jRDD = sc.parallelize(jDataList)
        jDataFrame = spark.read.json(jRDD)
        return jDataFrame
    else:
        print("URL request failed, error code: " + str(myResponse.status_code))
        

def mountStorage(accountName, containerName, folderName, mountPoint, scopeName, applicationIDKeyName, clientIDKeyName, tenantIDKeyName):
    """
    Mount the ADLS gen 2 storage file
    
    Returns: None
    """
    # Application (Client) ID
    applicationId = dbutils.secrets.get(scope=scopeName,key=applicationIDKeyName)

    # Application (Client) Secret Key
    authenticationKey = dbutils.secrets.get(scope=scopeName,key=clientIDKeyName)

    # Directory (Tenant) ID
    tenantId = dbutils.secrets.get(scope=scopeName,key=tenantIDKeyName)

    endpoint = "https://login.microsoftonline.com/" + tenantId + "/oauth2/token"
    source = "abfss://" + containerName + "@" + accountName + ".dfs.core.windows.net/" + folderName

    # Connecting using Service Principal secrets and OAuth
    configs = {"fs.azure.account.auth.type": "OAuth",
               "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
               "fs.azure.account.oauth2.client.id": applicationId,
               "fs.azure.account.oauth2.client.secret": authenticationKey,
               "fs.azure.account.oauth2.client.endpoint": endpoint}

    # Mounting ADLS Storage to DBFS
    # Mount only if the directory is not already mounted
    if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
            source=source,
            mount_point=mountPoint,
            extra_configs=configs)
    else:
        print("We have already mounted the mount point " + mountPoint)
        

def unmountStorage(mountPoint):
    """
    Unmount the ADLS gen 2 storage file
    
    Returns: None
    """
    # Unmount only if directory is mounted
    if any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(mountPoint)
    else:
        print("The mount point " + mountPoint + " isn't currently mounted")

        
def writeDfToDeltaTable(dataframe, databaseName, tableName, writeMode, partitionColumn):
    """
    Write Spark Data Frame to Delta table
    
    Returns: None
    """
    location = databaseName + "." + tableName
    dataframe.write.format("delta").mode(writeMode).partitionBy(partitionColumn).saveAsTable(location)
    
    
def writeDfToStorage(dataFrame, storagePath):
    """
    Write Spark Data Frame to Storage Account
    
    Returns: None
    """
    dataFrame.write.format('csv').mode('overwrite').option('header', True).save(storagePath)
    
    
def getCurrentTimeUTC():
    """
    Get the current time in UTC in the format yyyymmddhh
    
    Returns: Current time in UTC (in format yyyymmddhh) in string format
    
    """
    return datetime.utcnow().strftime('%Y%m%d%H')


def getCurrentYearUTC():
    """
    Get the current year
    
    Returns: Current year (in format yyyy) in string format
    
    """
    return datetime.utcnow().strftime('%Y')


def getCurrentMonthUTC():
    """
    Get the current month
    
    Returns: Current month (in format mm) in string format
    
    """
    return datetime.utcnow().strftime('%m')


def getCurrentDayUTC():
    """
    Get the current day
    
    Returns: Current month (in format dd) in string format
    
    """
    return datetime.utcnow().strftime('%d')
  
    
def getLastNDays(numDaysBack):
    """
    Get a list of the last N days
    
    Returns: A list of the last N dates (in format yyyy-mm-dd)
    
    """
    dayList = []
    for counter in range(0, numDaysBack):
        date = (datetime.utcnow() - timedelta(days=counter)).strftime('%Y-%m-%d')
        dayList.append(date)
    return dayList


def getLastNHrs(numHrsBack):
    """
    Get a list of the last N hours
    
    Returns: A list of the last N hours (in format yyyy-mm-dd hh:00)
    
    """
    hrList = []
    for counter in range(0, numHrsBack):
        date = (datetime.utcnow() - timedelta(hours=counter)).strftime('%Y-%m-%d %H:00')
        hrList.append(date)
    return hrList
  
    
def getnextNHrs(numHrsFwd):
    """
    Get a list of the next N hours
    
    Returns: A list of the next N hours (in format yyyy-mm-dd hh:00)
    
    """
    hrList = []
    for counter in range(0, numHrsFwd):
        date = (datetime.utcnow() + timedelta(hours=counter)).strftime('%Y-%m-%d %H:00')
        hrList.append(date)
    return hrList


def clearBronzeDatabase():
    listRawTables = spark.catalog.listTables("bronze")
    for table in listRawTables:
        name = table[0]
        fullName = "bronze." + name
        spark.sql("DROP TABLE {}".format(fullName))
    print("Bronze Database Cleared!")
    return
